# Résolution de problème d'optimisation linéaire (mixte) à l'aide de CBC et Python-MIP

Il est possible de résoudre des problèmes d'optimisation linéaire (en variables continues et/ou entières) en utilisant le solveur [CBC](https://github.com/coin-or/Cbc) via le modeleur [Python-MIP](https://www.python-mip.com/).

Il faut commencer par installer le package Python-MIP en exécutant la commande suivante.

In [7]:
%pip install mip

Note: you may need to restart the kernel to use updated packages.


On peut alors importer les définitions du module `model` du package mip.

In [1]:
from mip import *

## Création d'un modèle simple

Voici comment résoudre le problème linéaire mixte suivant

$$
\begin{align}
&\min 3 x + 2 y + z\\
&3 x + 4 y + 2 z \ge 12\\
&x +  y + z = 5\\
&x \ge 0\\
&y\text{ free}\\
&z \in \mathbb{Z}
\end{align}
$$


On crée d'abord un modèle contenant le problème à résoudre. Pour définir un modèle, on utilise la classe `Model`.

In [8]:
m = Model()
m.verbose = 0 # pour qu'il n'affiche aucun commentaire pendant la résolution

Pour déclarer une variable, on utilise la méthode `m.add_var`. Si l'on souhaite créer plusieurs variables, on peut créer un tableau contenant une variable par case.

**Remarques :**

- Il est possible de spécifier les bornes inférieure et supérieure pour chaque variable avec les paramètre `lb` et `ub`.

  **Attention :** par défaut, la borne inférieure est 0. Il faut metre `lb=-INF` (car `INF` est défini dans le module `mip`)  `lb=float('-inf')` pour qu'une variable n'ait pas de borne inférieure.
- On peut spécifier que la variable est de type entière (paramètre `var_type=INTEGER`) ou binaire (paramètre `var_type=BINARY`).
- Il est possible de donner un nom à une variable pour simplifier la lecture des fichiers au format lp.

In [10]:
x = m.add_var()
y = m.add_var(lb = -INF)
z = m.add_var(lb = -INF, var_type=INTEGER)

Pour la fonction objective, on fixe la valeur de `m.objective` égale à une fonction linéaire que l'on optimise (`minimize` ou `maximize`).

In [5]:
m.objective = minimize(3 * x + 2 * y + z)

Chaque contrainte est ajoutée en utilisant `m += `. La contrainte peut être définie par exemple avec `xsum`. Le signe est `<=`, `>=` ou `==`.

In [6]:
m += 3 * x + 4 * y + 2 * z >= 12
m += x + y + z == 5

## Optimisation et affichage de la solution

On optimise le modèle avec `m.optimize()`. On récupère l'optimum avec `m.objective_value` et la solution optimale trouvée en utilisant la méthode `x` pour chaque variable.

In [7]:
m.optimize()
print(f"optimum = {m.objective_value}")
print(f"x = {x.x}")
print(f"y = {y.x}")
print(f"z = {z.x}")


optimum = 6.0
x = 0.0
y = 1.0
z = 4.0


## Création d'un modèle de grande taille

Il est possible de générer des modèles de grande taille (avec beaucoup de variables et de contraintes). Pour cela, on peut déclarer des tableaux de variables (tableaux où chaque case contient une variable). On accède alors aux variables avec les indices.

In [8]:
#Nouveau modèle
m = Model()

# Pour créer 5 variables binaires
J = range(5)
x = [m.add_var(var_type=BINARY) for j in J]

# On peut maintenant accéder aux variables avec x[0], x[1], x[2], x[3], x[4]

Pour écrire des contraintes ou la fonction objective, on peut utiliser la fonction `xsum` qui permet de faire des sommes en indiquant les indices sur lesquelles sont faites les sommes. Les contraintes peuvent être créées au sein d'une boucle.

Par exemple, supposons que l'on veuille résoudre le problème suivant : 

$$
\begin{align}
& \max 3 x_0 + 4 x_1 + 7 x_2 + 8 x_3 + 10 x_4\\
& x_0 + 2 x_1 + 4 x_2 + 6 x_3 + 7 x_4 \le 10\\
& x_j \in \{0,1\} \quad \text{for $j = 1,\dots,5$}
\end{align}
$$

In [11]:
# On souhaite minimiser 3 x[0] + 4 x[1] - 5 x[2]
c = [3, 4, 7, 8, 10]
a = [1, 2, 4, 6, 7]
b = 10

# Définition de l'objectif
m.objective = maximize(xsum(c[j] * x[j] for j in J))

# Ajout de la seule contrainte linéaire
m += xsum(a[j] * x[j] for j in J) <= b

m.verbose = 0
m.optimize()
print(f"optimum = {m.objective_value}")
solution = [x[j].x for j in J]

for j in J:
    print(f"- x[{j}] = {solution[j]}")

optimum = 17.0
- x[0] = 1.0
- x[1] = 1.0
- x[2] = 0.0
- x[3] = 0.0
- x[4] = 1.0


Pour plus d'information sur le package Python-MIP, vous pouvez consulter la [documentation](https://docs.python-mip.com/en/latest/index.html). 